## Cyclic ID Algorithm Implementation with Examples

##### Based on: Forré & Mooij (2019) - "Causal Calculus in the Presence of Cycles, Latent Confounders and Selection Bias"

**Paper Link:** [arXiv:1901.00433](https://arxiv.org/abs/1901.00433) | [PDF](https://arxiv.org/pdf/1901.00433.pdf)

**Citation:**
```
Forré, P., & Mooij, J. M. (2019). Causal calculus in the presence of cycles, 
latent confounders and selection bias. arXiv preprint arXiv:1901.00433.
```

### 1. Introduction & Setup

#### 1.1 Why do we need an algorithm for Cyclic Graphs?

Traditional causal inference methods assume **acyclic graphs (DAGs)**, which have no feedback loops allowed in them. However, more realistic systems and networks often do include these cycles. Some examples include:

* Gene regulatory networks (GRNs): Gene X activates gene Y, which activates gene Z, which feeds back to activate gene X again. 

* Economic systems: Interest rates affect inflation, which affects interest rates.

* Neural networks: Recurrent connections create feedback loops.

The implementation of this **generalized ID** algorithm for input-output Structural Causal Models (ioSCMs) handles:

* Cycles (feedback loops)

* Latent confounders (unmeasured causes)

* Arbitrary probability distributions


#### 1.2 Key Question

Given a directed mixed graph $G$ with cycles, observational data $P(V)$ and a causal query: "What is $P(Y | \text{do}(X))$?

The main question we ask with this algorithm is ***can we identify the causal effect $P(Y | \text{do}(X))$ from the observational distribution $P(V)$ alone?***

The **generalized ID algorithm** answers this question for cyclic graphs. 


In [12]:
# importing packages and y0 module

from IPython.display import Markdown, display

from y0.algorithm.ioscm.utils import (
    get_apt_order,
    get_consolidated_district,
    get_graph_consolidated_districts,
    get_strongly_connected_components,
    get_vertex_consolidated_district,
    is_apt_order,
)
from y0. dsl import P, Variable
from y0.graph import NxMixedGraph

#### 1.3 Example Graphs to Demonstrate the Need for this algorithm

Here are two examples that illustrate why we need cyclic identification. 

#### Example 1: Simple Feedback Loop

A simple non-identifiable cyclic structure:

<div align="center">
    <img src="./feedback_loop1.png" alt="Feedback Loop Simple Graph Example" width="400">
</div>


**This is an example of the simplest cyclic structure. Two variables that affect each other simultaneously.**

- **Graph structure:** $x \rightarrow y$ and $y \rightarrow x$
- **Strongly Connected Component:** {x, y} form one SCC since each can reach the other.


**The fundamental problem:** 
When we observe data P(x,y), we see their joint behavior under the feedback loop. But if we want to answer the causal question "What happens to y if I intervene and set x to a specific value?" - we need P(y | do(x)). 

**Key Question:** Can we identify P(y | do(x)) from observational data P(x,y) alone?

#### Example 2: More Complex Graph: Cycles and Confounders

<div align="center">
    <img src="./graph 2 with confounders.png" alt="Example Graph with Confounders" width="400">
</div>

**Complex cyclic structure with hidden causes:**

- **Directed cycle:** x → y → z → x (feedback loop)
- **Latent confounder:** x ↔ z (unmeasured common cause U)
- **Key challenges:** This graph has both feedback AND unmeasured confounding
- **Identifiable?** This depends on the structure; the generalized ID algorithm determines if this kind of graph is identifiable or not. (Success or Fail)

### 2. Working Examples: Using a Directed Mixed Graph (DMG) 

For this specific implementation, we will walkthrough the cyclic ID algorithm using a directed mixed graph (DMG) that exhibits some key challenges:

1. **Cycles** (feedback loops)
2. **Latent confounders** (unmeasured common causes)
3. **Self-loops**

#### 2.1 Graph Structure - Success Case 

<div align="center">
    <img src="./DMG_example_1success.png" alt="Example DMG with cycles and self-loops" width="400">
</div>

This example graph contains:

**Variables:** {𝒳, 𝒴, 𝒵, 𝒲}

**Directed edges:**
- 𝒲 → 𝒳 (external input - potential intervention variable)
- 𝒳 → 𝒳 (self-loop - auto-regulatory feedback)
- 𝒳 → 𝒴 (forward edge)
- 𝒴 → 𝒵 (forward edge)
- 𝒵 → 𝒳 (feedback edge - completes 3-node cycle)

**Key structural features:**
1. **3-node cycle:** 𝒳 → 𝒴 → 𝒵 → 𝒳 (feedback loop)
2. **Self-loop:** 𝒳 → 𝒳 (auto-regulation)
3. **External input:** 𝒲 → 𝒳 (entry point for interventions)
4. **No latent confounders** in this version (makes it identifiable)

**Causal Query:** Can we identify P(𝒴 | do(𝒲))? 
- **Effect:** How does intervening on 𝒲 affect outcome 𝒴?
- **Expected Result:** SUCCESS ✓ (the algorithm will identify this effect)


**Note:** We will start with an **identifiable (SUCCESS)** case to understand how the algorithm works step-by-step. In further sections we will do a partially identifiable case, and a failure case. 


In [13]:
# Creating a success case graph to demonstrate the algorithm

from y0.dsl import Variable
from y0.graph import NxMixedGraph

# define the variables needed
x = Variable("x")
y = Variable("y")
z = Variable("z")
w = Variable("w")

# initialize the DMG (directed mixed graph)

graph_case_success = NxMixedGraph()

# add the directed edges
graph_case_success.add_directed_edge(w, x) # w -> x
graph_case_success.add_directed_edge(x, x) # x -> x (self-loop)
graph_case_success.add_directed_edge(x, y) # x -> y
graph_case_success.add_directed_edge(y, z) # y -> z
graph_case_success.add_directed_edge(z, w) # z -> w (completes the cycle)

# display the graph structure
print("Graph structure (success case):")
print(f"\nNodes: {sorted([str(n) for n in graph_case_success.nodes()])}")

print(f"\nDirected Edges: ({len(list(graph_case_success.directed.edges()))})")
for edge in sorted(graph_case_success.directed.edges(), key=lambda e: (str(e[0]), str(e[1]))):
    print(f"  {edge[0]} -> {edge[1]}")
    
print(f"\nBidirected Edges: ({len(list(graph_case_success.undirected.edges()))})")



Graph structure (success case):

Nodes: ['w', 'x', 'y', 'z']

Directed Edges: (5)
  w -> x
  x -> x
  x -> y
  y -> z
  z -> w

Bidirected Edges: (0)


#### 2.2 Algorithm Execution

##### Line 1: Function Declaration

**Algorithm:**
```
function ID(G, Y, W, P(V | do(J)))
```

**What this line does:** 
Declares the ID function with its inputs.

**Our specific inputs:**
- `G` = graph_case_success (our DMG)
- `Y` = {y} (target variable)
- `W` = {w} (intervention variable)
- `J` = ∅ (no background interventions)
- `P(V | do(J))` = P(V) (observational distribution, assuming we have data)

> **For detailed explanation of each parameter**, see the Explanation Notebook Line 1.

Let's set up our inputs in code:


In [14]:
# This cell is setting up the cyclic ID algorithm inputs

# Using the graph creeated in Section 2.1
G = graph_case_success

# Define our causal query
Y = {y}  # Target: we want to know the effect on y
W = {w}  # Intervention: we're intervening on w
J = set()  # No background interventions (empty set)

# Display our setup
print("=" * 60)
print("Algorithm 1: ID Function - Line 1")
print("=" * 60)
print(f"\nGraph G:")
print(f"   Nodes: {sorted([str(v) for v in G.nodes()])}")
print(f"   Edges: {len(list(G.directed.edges()))} directed")

print(f"\nCausal Query Setup:")
print(f"   Y (target):       {{{', '.join(str(v) for v in Y)}}}")
print(f"   W (intervention): {{{', '.join(str(v) for v in W)}}}")
print(f"   J (background):   {J if J else '∅ (empty)'}")

print(f"\nQuestion: Can we identify P(Y | do(W))?")
print(f"   In plain English: What is the causal effect of intervening")
print(f"   on {list(W)[0]} on the outcome {list(Y)[0]}?")
print("\n" + "=" * 60)

Algorithm 1: ID Function - Line 1

Graph G:
   Nodes: ['w', 'x', 'y', 'z']
   Edges: 5 directed

Causal Query Setup:
   Y (target):       {y}
   W (intervention): {w}
   J (background):   ∅ (empty)

Question: Can we identify P(Y | do(W))?
   In plain English: What is the causal effect of intervening
   on w on the outcome y?



##### Line 2: Precondition Checks

**Algorithm:**
```
require: Y ⊆ V, W ⊆ V, Y ∩ W = ∅
```
**What this line does:** 
Validates that the inputs are valid before proceeding with the algorithm.

If any condition fails, the algorithm terminates immediately. Let's verify these conditions for our example:


In [22]:
# This cell covers Line 2 of the algorithm to check the preconditions

V = set(G.nodes()) # all observed variables in the graph since this algorithm takes only observed variables

print("=" * 60)
print("Algorithm 1: ID Function - Line 2")
print("=" * 60)

# Check 1: Is Y a subset of V? (target variables are in graph)
check1 = Y.issubset(V)
print(f"\n✓ Check 1: Y ⊆ V (targets must be in graph)")
print(f"   Y = {{{', '.join(str(v) for v in sorted(Y, key=str))}}}")
print(f"   V = {{{', '.join(str(v) for v in sorted(V, key=str))}}}")
print(f"   Y ⊆ V? {check1} {'✓ PASS' if check1 else '✗ FAIL'}")

# Check 2: Is W a subset of V? (intervention variables are in graph)
check2 = W.issubset(V)
print(f"\n✓ Check 2: W ⊆ V (interventions must be in graph)")
print(f"   W = {{{', '.join(str(v) for v in sorted(W, key=str))}}}")
print(f"   V = {{{', '.join(str(v) for v in sorted(V, key=str))}}}")
print(f"   W ⊆ V? {check2} {'✓ PASS' if check2 else '✗ FAIL'}")

# Check 3: Y ∩ W = ∅
intersection = Y.intersection(W)
check3 = len(intersection) == 0
print(f"\n✓ Check 3: Y ∩ W = ∅ (no overlap allowed")
if intersection:
    print(f"   Y ∩ W = {{{', '.join(sorted([str(v) for v in intersection], key=str))}}}")
else:
    print(f"   Y ∩ W = ∅ (empty)")
print(f"   Disjoint? {check3} {'✓ PASS' if check3 else '✗ FAIL'}")
    
# Overall result
all_checks_pass = check1 and check2 and check3
print(f"\n{'='*60}")
if all_checks_pass:
    print("All preconditions satisfied")
    print("   → Can proceed to Line 3.")
else:
    print("Preconditions failed. Algorithm cannot proceed.")
print("=" * 60)



Algorithm 1: ID Function - Line 2

✓ Check 1: Y ⊆ V (targets must be in graph)
   Y = {y}
   V = {w, x, y, z}
   Y ⊆ V? True ✓ PASS

✓ Check 2: W ⊆ V (interventions must be in graph)
   W = {w}
   V = {w, x, y, z}
   W ⊆ V? True ✓ PASS

✓ Check 3: Y ∩ W = ∅ (no overlap allowed
   Y ∩ W = ∅ (empty)
   Disjoint? True ✓ PASS

All preconditions satisfied
   → Can proceed to Line 3.


Note: Since the preconditions have all passed, this means the algorithm can continue on to the further steps to identify the causal effects.

##### Line 3 - Computing the Ancestral Closure H

**Algorithm:**
```
H ← Anc^(G_(V\W))(Y)
```
**What this line does:**
Computes the **ancestral closure** of target variables Y after removing intervention variables W from the graph.

**Two-step process:**
1. **Graph surgery**: Create mutilated graph `G_(V\W)` by removing all nodes in W
2. **Find ancestors**: Compute all ancestors of Y in the mutilated graph

**Why this matters:**
Variables that are not ancestors of Y (after removing W) are irrelevant for identifying the causal effect. This step focuses the algorithm on only the causally relevant variables.

> **For detailed explanation**, see the Explanation Notebook Line 3.

Let's compute H for our example:

In [26]:
# This cell will demonstrate how to compute the ancestral closure H

print("=" * 60)
print("Algorithm 1: Line 3 - Compute Ancestral Closure H")
print("=" * 60)


# step 1: Graph surgery - remove W and its outgoing edges
print(f"\nStep 1: Graph Surgery")
print(f"   Original nodes V: {sorted([str(v) for v in V], key=str)}")
print(f"   Remove W: {{{', '.join(sorted([str(v) for v in W], key=str))}}}")

# create a set of nodes to keep (V \ W)
V_minus_W = V - W
print(f"   Remaining (V\\W): {sorted([str(v) for v in V_minus_W], key=str)}")

# create a mutilated graph by keeping only nodes in V \ W
G_mutilated = G.subgraph(V_minus_W)
print(f"   Mutilated graph: {len(G_mutilated.nodes())} nodes, {len(list(G_mutilated.directed.edges()))} directed edges")


# Step 2: Find ancestors of Y in the mutilated graph
print(f"\nStep 2: Find Ancestors of Y")
print(f"   Target Y = {{{', '.join(sorted([str(v) for v in Y], key=str))}}}")



Algorithm 1: Line 3 - Compute Ancestral Closure H

Step 1: Graph Surgery
   Original nodes V: ['w', 'x', 'y', 'z']
   Remove W: {w}
   Remaining (V\W): ['x', 'y', 'z']
   Mutilated graph: 3 nodes, 3 directed edges

Step 2: Find Ancestors of Y
   Target Y = {y}
